# Tokenização e Lematização

In [21]:
#%pip install nltk spacy

In [22]:
# Importando as bibliotecas
import pandas as pd
import string
import nltk
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [23]:
# Baixar recursos
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\josaf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\josaf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
#!python -m spacy download pt_core_news_sm

In [25]:
# Carrega modelo SpaCy
nlp = spacy.load("pt_core_news_sm")

In [26]:
# Função de pré-processamento com lematização
stopwords_pt = set(stopwords.words('portuguese'))

def preprocessar_com_lematizacao(texto):
  texto = texto.lower()
  texto = texto.translate(str.maketrans('', '', string.punctuation))
  doc = nlp(texto)
  tokens = [token.lemma_ for token in doc if token.text not in stopwords_pt and not token.is_space]
  return ' '.join(tokens)

In [27]:
# Função de pré-processamento sem lematização
def preprocessar_sem_lematizacao(texto):
  texto = texto.lower()
  texto = texto.translate(str.maketrans('','',string.punctuation))
  tokens = word_tokenize(texto, language='portuguese')
  tokens = [word for word in tokens if word not in stopwords_pt]
  return ' '.join(tokens)

In [28]:
# Carregar dataset
df = pd.read_csv('../../datasets/dataset_exemplo_nlp.csv')

In [29]:
# Aplicar os dois tipos de pré-processamento
df['texto_sem_lematizacao'] = df['texto'].apply(preprocessar_sem_lematizacao)
df['texto_com_lematizacao'] = df['texto'].apply(preprocessar_com_lematizacao)

In [30]:
# Vetorização
vectorizer = CountVectorizer()
X_sem = vectorizer.fit_transform(df['texto_sem_lematizacao'])
X_com = vectorizer.fit_transform(df['texto_com_lematizacao'])
y = df['classe']

In [31]:
# Divisão treino/teste
X_train_sem, X_test_sem, y_train, y_test = train_test_split(X_sem, y, test_size=0.3, random_state=42)
X_train_com, X_test_com, _,_ = train_test_split(X_com, y, test_size=0.3, random_state=42)

In [32]:
# Modelo sem lematização
modelo_sem = LogisticRegression()
modelo_sem.fit(X_train_sem, y_train)
y_pred_sem = modelo_sem.predict(X_test_sem)

In [33]:
# Modelo com lematização
modelo_com = LogisticRegression()
modelo_com.fit(X_train_com, y_train)
y_pred_com = modelo_com.predict(X_test_com)

In [34]:
# Resultados
print('Desempenho SEM lematização')
print(classification_report(y_test, y_pred_sem))

Desempenho SEM lematização
              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00        26
    positivo       1.00      1.00      1.00        34

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



In [35]:
print('Desempenho COM lematização')
print(classification_report(y_test, y_pred_com))

Desempenho COM lematização
              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00        26
    positivo       1.00      1.00      1.00        34

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



In [43]:
# Testar frase nova
frase_nova = "Achei o atendimento muito demorado e desorganizado"

# Pré-processar com e sem lematização
frase_sem_lematizacao = preprocessar_sem_lematizacao(frase_nova)
frase_com_lematizacao = preprocessar_com_lematizacao(frase_nova)

# Vetorizar (usando o mesmo vectorizer treinado anteriormente!)
X_nova_sem = vectorizer.transform([frase_sem_lematizacao])
X_nova_com = vectorizer.transform([frase_com_lematizacao])

# Fazer previsão
pred_sem = modelo_sem.predict(X_nova_sem)[0]
pred_com = modelo_com.predict(X_nova_com)[0]

print(f"\nFrase: {frase_nova}")
print(f"→ Classificação SEM lematização: {pred_sem}")
print(f"→ Classificação COM lematização: {pred_com}")



Frase: Achei o atendimento muito demorado e desorganizado
→ Classificação SEM lematização: negativo
→ Classificação COM lematização: positivo
